# Dataset

I've picked up 4 articles from medium.com. The first two are related to NLP, while the third and fourth are related to the political climate in Russia, Ukraine and China.

In [ ]:
%%capture
!wget https://raw.githubusercontent.com/sarthakrastogi/NLP-Experiments/main/Article%20Similarity/NLP1.txt
!wget https://raw.githubusercontent.com/sarthakrastogi/NLP-Experiments/main/Article%20Similarity/NLP2.txt
!wget https://raw.githubusercontent.com/sarthakrastogi/NLP-Experiments/main/Article%20Similarity/RC1.txt
!wget https://raw.githubusercontent.com/sarthakrastogi/NLP-Experiments/main/Article%20Similarity/RU2.txt

In [ ]:
with open('NLP1.txt', 'r') as f:
    NLP1 = f.read()
with open('NLP2.txt', 'r') as f:
    NLP2 = f.read()
with open('RC1.txt', 'r') as f:
    RC1 = f.read()
with open('RU2.txt', 'r') as f:
    RU2 = f.read()

articles = [BERT1, BERT2, RC1, RU2]

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
import numpy as np

import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Stopword removal

In [ ]:
stop_words_l=stopwords.words('english')
cleaned_articles = []
for x in articles:
  cleaned_articles.append(" ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l))
articles = cleaned_articles


Decontracting words

In [ ]:
import re

def decontracted(phrase):
    """already used and applied to the data"""

    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase


# Doc2Vec

In [ ]:
!pip install gensim

In [ ]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec

In [ ]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(articles)]

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=30, min_count=2, epochs=80)

In [ ]:
model.build_vocab(tagged_data)

In [ ]:
model.train(tagged_data, total_examples=model.corpus_count, epochs=80)

In [ ]:
model.save("d2v.model")

In [ ]:
model = Doc2Vec.load("d2v.model")

In [ ]:
similar_doc = model.docvecs.most_similar('0')
print(similar_doc[0])

('1', 0.39299920201301575)


In [ ]:
model.docvecs.similarity(0, 3)

0.26954013

In [ ]:
vectors = [model.docvecs[i] for i in range(4)]

In [ ]:
pairwise_similarities=cosine_similarity(vectors)
pairwise_similarities

array([[0.9999998 , 0.39299917, 0.23008074, 0.26954013],
       [0.39299917, 1.0000001 , 0.2833395 , 0.2238168 ],
       [0.23008074, 0.2833395 , 1.        , 0.07627331],
       [0.26954013, 0.2238168 , 0.07627331, 0.99999994]], dtype=float32)

3 and 4 are very different from 1 and 2, and they have correctly been given low scores (less than 0.3). But while articles 1 and 2 are quite similar, their similarity score 0.39 is quite low.

# BERT

In [ ]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')


     |████████████████████████████████| 79 kB 1.6 MB/s 
     |████████████████████████████████| 3.8 MB 16.8 MB/s 
     |████████████████████████████████| 1.2 MB 46.0 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 895 kB 45.8 MB/s 
     |████████████████████████████████| 596 kB 49.0 MB/s 
     |████████████████████████████████| 6.5 MB 39.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=1003fc91c75578ac4c1a7c33fd2ebbcbe84cf4d564a0bf35ecfe00d1e5e338e4
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
document_embeddings = sbert_model.encode(articles)
pairwise_similarities = cosine_similarity(document_embeddings)

In [ ]:
pairwise_similarities

array([[0.99999976, 0.73256826, 0.53020215, 0.6046282 ],
       [0.73256826, 1.        , 0.6555556 , 0.6972009 ],
       [0.53020215, 0.6555556 , 0.9999999 , 0.76593095],
       [0.6046282 , 0.6972009 , 0.76593095, 0.99999994]], dtype=float32)